#Classification problem using AutoML H20

In [7]:
#Data management
import pandas as pd
import joblib

#autoMachineLearning
import h2o
from h2o.automl import H2OAutoML
from h2o.grid.grid_search import H2OGridSearch

In [8]:
train = pd.read_csv('preprocessed_df.csv',index_col=0) #Loading TRAIN data

In [9]:
train

,SeniorCitizen,Partner,Dependents,tenure,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,0,1,0,0.013889,0,0,2,0,0,0,1,2,0.115423,0
1,0,0,0,0.472222,0,2,0,2,0,1,0,3,0.385075,0
2,0,0,0,0.027778,0,2,2,0,0,0,1,3,0.354229,1
3,0,0,0,0.625000,0,2,0,2,2,1,0,0,0.239303,0
4,0,0,0,0.027778,1,0,0,0,0,0,1,2,0.521891,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,0.333333,0,2,0,2,2,1,1,3,0.662189,0
7039,0,1,1,1.000000,1,0,2,2,0,1,1,1,0.845274,0
7040,0,1,1,0.152778,0,2,0,0,0,0,1,2,0.112935,0
7041,1,1,0,0.055556,1,0,0,0,0,0,1,3,0.558706,1


In [10]:
h2o.init() #To start h2o

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 22 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_Nicolas_7d3fed
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.448 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [11]:
h2train = h2o.H2OFrame(train) #H20Frame is equal to a Pandas dataframe. We parse so it can be later trained.

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
cat_cols = train.columns.drop([ 'tenure', 'MonthlyCharges'])

for col in cat_cols:
    h2train[col] = h2train[col].asfactor()



In [15]:
x = list(train.columns)
x.remove('Churn')

y = "Churn"

print("X:", x)
print("y:", y)

X: ['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges']
y: Churn


In [18]:
#TRAINING all the h20 models

automl = H2OAutoML(max_models=30,max_runtime_secs=3600, sort_metric='f1',  balance_classes=True)
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |█
12:23:28.776: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_3_AutoML_2_20230314_122328_model_2


Status of Neuron Layers: predicting Churn, 2-class classification, bernoulli distribution, CrossEntropy loss, 1,762 weights/biases, 27.2 KB, 15,900,734 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms          mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  ------------------  -------------------  ------------------
    1        43       Input             5.0
    2        20       RectifierDropout  0.0        0.0   0.0   0.26095493227990646    0.4378516674041748     0.0         0.015894076205030888   0.8397924900054932  -1.4913680543733665  2.347810745239258
    3        20       RectifierDropout  0.0        0.0   0.0   0.0017623920794358127  0.001310055609792471   0.0         -0.2484694400254375    1.208451271057129   -0.5857974388469798  2.385993003845215
    4        20       RectifierDropout  0.0        0.0   0.0   0.0029183721925983264  0.006753856316208839   0.0         -0.384113822139916     1.0039324760437012  -1.631430896224564   1.636793613433838
    5        2        Softmax                      0.0   0.0   0.002375570376170799   0.0014663147740066051  0.0         -0.003916765982285142  1.0964608192443848  0.7061139066063116   0.2972235679626465

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.1472837850561023
RMSE: 0.3837756962811771
LogLoss: 0.45090278831799663
Mean Per-Class Error: 0.16013118562098969
AUC: 0.9139863662378187
AUCPR: 0.8986435068070201
Gini: 0.8279727324756374

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1602555065493219
       0     1     Error    Rate
-----  ----  ----  -------  ----------------
0      3808  1190  0.2381   (1190.0/4998.0)
1      411   4591  0.0822   (411.0/5002.0)
Total  4219  5781  0.1601   (1601.0/10000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.160256     0.851526  321
max f2                       0.0315044    0.918341  375
max f0point5                 0.420588     0.830211  232
max accuracy                 0.162903     0.8402    320
max precision                0.999282     1         0
max recall                   0.000554014  1         398
max specificity              0.999282     1         0
max absolute_mcc             0.162903     0.688337  320
max min_per_class_accuracy   0.345651     0.82513   258
max mean_per_class_accuracy  0.162903     0.84017   320
max tns                      0.999282     4998      0
max fns                      0.999282     4982      0
max fps                      3.20042e-05  4998      399
max tps                      0.000554014  5002      398
max tnr                      0.999282     1         0
max fnr                      0.999282     0.996002  0
max fpr                      3.20042e-05  1         399
max tpr                      0.000554014  1         398

Gains/Lift Table: Avg response rate: 50.02 %, avg score: 35.88 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101                      0.98819            1.9992     1.9992             1                0.995012     1       

In [17]:
#Showing the best performers

leader_board = automl.leaderboard
leader_board.head()

model_id,aucpr,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20230314_120305,0.663626,0.847154,0.414123,0.234934,0.366652,0.134434
StackedEnsemble_BestOfFamily_1_AutoML_1_20230314_120305,0.663452,0.84663,0.414949,0.23575,0.367108,0.134768
GBM_5_AutoML_1_20230314_120305,0.661782,0.844228,0.418246,0.236452,0.368527,0.135812
GBM_1_AutoML_1_20230314_120305,0.661075,0.844951,0.416573,0.235235,0.367866,0.135325
GBM_grid_1_AutoML_1_20230314_120305_model_1,0.660151,0.845962,0.415486,0.235973,0.367211,0.134844
GBM_grid_1_AutoML_1_20230314_120305_model_7,0.658712,0.84398,0.418286,0.231491,0.368689,0.135932
GBM_grid_1_AutoML_1_20230314_120305_model_2,0.657539,0.844449,0.418144,0.235737,0.368731,0.135962
GBM_2_AutoML_1_20230314_120305,0.657059,0.840008,0.423534,0.238779,0.371129,0.137737
GBM_grid_1_AutoML_1_20230314_120305_model_4,0.654453,0.843076,0.42036,0.235897,0.369709,0.136684
GBM_grid_1_AutoML_1_20230314_120305_model_8,0.654356,0.843677,0.419171,0.2306,0.369287,0.136373
